# On modelling optimization problems via Julia JuMP

## Prof. Mayron César O. Moreira 

**Universidade Federal de Lavras (UFLA)**  
**Department of Computer Science**  
**Lavras, Minas Gerais, Brazil**  

*Università degli Studi di Modena e Reggio Emilia (UNIMORE)*  
*Reggio Emilia, Italy*

## Maximum Dispersion Problem

Let $V=\{1,...,n\}$ be a set of objects and $d_{ij}$ be the similarity distance between objects $i,j \in V$. We take $d_{ii}=0, \forall i \in V$. Define $a_i \ge 0$ as the weight or attribute of object $i \in V$. Consider a set of groups $C=\{1,...,m\}$ and a target weight $\mathcal{M}_k \ge 0$, for each group $k \in C$. We allow a deviation up to $\alpha \ge 0$ for the parameter $\mathcal{M}_k$ such that the sum of weights of each group must lie in the interval $[(1-\alpha)\mathcal{M}_k, (1+\alpha)\mathcal{M}_k]$. The objective consists in **maximize the minimum pairwise distance** between two objects assigned to the same group.

Fernández et al. (2013) studied this problem, called **Maximum Dispersion Problem** (MaxDP), and propose the two formulations we will see this in this notebook. In that paper, the authors cite Baker & Powell (2002) study as a motivation for the MaxDP. According to Baker and Powell (2002),  a solution of the MaxDP creates groups of students that prioritize heterogeneity of academic backgrounds, which can contribute to the learning process.

* **Importing libraries**

In [1]:
include("codes/instanceMaxDP.jl")
; # Disable output messages after the block

* **Reading instance**

    - We focus on "*study type*" instances. Fernández et al. (2013) evaluate the distance between each student according to Likert (1932).

In [4]:
# MaxDP instance
fileInstance="instances/maxDP/study-100-4-000-1"

io=open(fileInstance)

# Reading MaxDP instance
instance = readMaxDP(io)
;

### Model 1 (M1)

* **Variables**
    * $x_{ik} \in \{0,1\}$: equals one if object $i$ is assigned to group $k$; zero, otherwise.  
    * $z_{ijk} \in \{0,1\}$: equals one if objects $i$ and $j$ are assigned to group $k$; zero, otherwise.
    * $u \ge 0$: max-min parwise distance.
    
Without loss of generality, it is assumed that $i < j$, to avoid symmetry due to $z_{ijk} = z_{jik}$. The model M1 reads as follows.

\begin{equation}
\max u
\end{equation}

subject to

\begin{alignat}{2}
\sum_{k \in C} x_{ik} = 1 &&  \qquad & \forall i \in V\\
\sum_{i \in V} a_ix_{ik} \ge (1-\alpha)\mathcal{M}_k &&  \qquad &\forall k \in C\\
\sum_{i \in V} a_ix_{ik} \le (1+\alpha)\mathcal{M}_k &&  \qquad &\forall k \in C\\
u \le d_{ij}z_{ijk} + D(1-z_{ijk})&& \quad &\forall i,j \in V, i < j, \forall k \in C\\
x_{ik} + x_{jk} \le 1 + z_{ijk}&& \quad &\forall i,j \in V, i < j, \forall k \in C\\
x_{ik} \in \{0,1\} && \quad &\forall i \in V, \forall k \in C\\
z_{ijk} \in \{0,1\} && \quad &\forall i,j \in V, i < j, \forall k \in C\\
u \ge 0.
\end{alignat}

* **Importing OR libraries**

In [5]:
# Importing libraries
using JuMP
using Cbc

* **Some parameters**

In [6]:
n = instance.n
m = instance.m

V = collect(1:n) # set of objects
C = collect(1:m) # set of groups
a = instance.a # weights
M = instance.M # target values
d = instance.d # distances
ud = instance.ud # unique values of distances (sorted in descending order)
alpha = 0.05 # tolerance {0.05; 0.01; 0.001}
;

* **Model**

    * Parameters: at most 60s, single thread, 10E-3 of allowable gap

In [7]:
model = Model(with_optimizer(Cbc.Optimizer, seconds=60, 
        allowableGap=1e-3,threads=1))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

* **Variables**

In [8]:
@variable(model, x[V,C], Bin)
@variable(model, z[V,V,C], Bin)
@variable(model, u >= 0)
;

* **Objective function**

In [9]:
@objective(model, Max, u)

u

* **Sets of constraints**

We take a sufficient large constant $D = \max_{i,j \in V} d_{ij}$

In [10]:
D = maximum(d)

61.0

* **Running the model**

In [12]:
@constraints(model, begin
        [i in V], sum(x[i,k] for k in C) == 1
        [k in C], sum(a[i]*x[i,k] for i in V) >= (1-alpha)*M[k]
        [k in C], sum(a[i]*x[i,k] for i in V) <= (1+alpha)*M[k]
        [i in V, j in V, k in C; i < j], 
            u <= d[i,j]*z[i,j,k] + D*(1 - z[i,j,k])
        [i in V, j in V, k in C; i < j], 
            x[i,k] + x[j,k] <= 1 + z[i,j,k]
        end)

optimize!(model)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Dec 31 2018 

command line - Cbc_C_Interface -threads 1 -seconds 60 -allowableGap 0.001 -solve -quit (default strategy 1)
threads was changed from 0 to 1
seconds was changed from 1e+100 to 60
allowableGap was changed from 1e-10 to 0.001
Continuous objective value is 61 - 0.47 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 19720 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 19764 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 19788 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 19776 strengthened rows, 0 substitutions
Cgl0004I processed model has 39688 rows, 20193 columns (20192 integer (20192 of which binary)) and 178808 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0030I Thread 0 used 0 times,  waiting to start 0.09664607, 0 cpu time, 0 locks, 0 locked, 0 waiting for locks
Cbc0030I Main thread 0 waiting for threads,  1 locks, 0 locked,

* **Printing variables**

In [14]:
objFunction = objective_value(model) # Optimal solution
for k in C
    print("Group ", k, ": ")
    aux = 0 # Auxiliary variable that counts the group load
    cnt = 0 # Counts the number of objects in each group
    for i in V
        if(value(x[i,k]) >= 0.9)
            print(i, " ")
            aux += a[i]
            cnt += 1
        end
    end
    println("(", aux, ") (", cnt, " objects)")
end

Group 1: (0) (0 objects)
Group 2: (0) (0 objects)
Group 3: (0) (0 objects)
Group 4: (0) (0 objects)


### Model 2 (M2)

* **Additional parameter**
    * $0 \le d^1 < d^2 < d^3 < ... < d^R = D$: distinct distances over all pair of objects in ascending order.

* **Variables**
    * $w^{r} \in \{0,1\}$: equals one if the overall smallest pairwise distance is at most $d^r$
   
The covering formulation, denoted by M2, reads as follows.

\begin{equation}
\max d^R + \sum_{r=1}^{R-1} (d^r - d^{r+1})w^r
\end{equation}

subject to

\begin{alignat}{2}
\sum_{k \in C} x_{ik} = 1 &&  \qquad & \forall i \in V\\
\sum_{i \in V} a_ix_{ik} \ge (1-\alpha)\mathcal{M}_k &&  \qquad &\forall k \in C\\
\sum_{i \in V} a_ix_{ik} \le (1+\alpha)\mathcal{M}_k &&  \qquad &\forall k \in C\\
x_{ik} + x_{jk} \le 1 + w^r&& \quad &\forall i,j \in V, i < j, \forall k \in C, 1 \le r \le R| d_{ij}=d^r\\
w^{r-1} \le w^r && \quad & 2 \le r \le R \\
x_{ik} \in \{0,1\} && \quad &\forall i \in V, \forall k \in C\\
w^r \in \{0,1\} && \quad & 1 \le r \le R.
\end{alignat}

Note that if we have $R=4$ and $w^2=1$, then $w^3 = w^4 = 1$, and the function will be given by: $(d^2 - d^3) + (d^3 - d^4) + d^4 = d^2$.

* **New parameters**

In [15]:
R, = size(ud) # number of distinct pairwise distances. "size" will return a tuple

(43,)

* **Model**

In [16]:
model = Model(with_optimizer(Cbc.Optimizer, seconds=60, 
        allowableGap=1e-3,threads=1))

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: COIN Branch-and-Cut (Cbc)

* **Variables**

In [17]:
@variable(model, x[V,C], Bin)
@variable(model, w[1:R], Bin)

43-element Array{VariableRef,1}:
 w[1] 
 w[2] 
 w[3] 
 w[4] 
 w[5] 
 w[6] 
 w[7] 
 w[8] 
 w[9] 
 w[10]
 w[11]
 w[12]
 w[13]
 ⋮    
 w[32]
 w[33]
 w[34]
 w[35]
 w[36]
 w[37]
 w[38]
 w[39]
 w[40]
 w[41]
 w[42]
 w[43]

* **Objective function**

In [19]:
@objective(model, Max, ud[R] + sum((ud[r] - ud[r+1])*w[r] for r=1:R-1))

-w[1] - 3 w[2] - w[3] - w[4] - w[5] - w[6] - w[7] - w[8] - w[9] - w[10] - w[11] - w[12] - w[13] - w[14] - w[15] - w[16] - w[17] - w[18] - w[19] - w[20] - w[21] - w[22] - w[23] - w[24] - w[25] - w[26] - w[27] - w[28] - w[29] - w[30] - w[31] - w[32] - w[33] - w[34] - w[35] - w[36] - w[37] - w[38] - w[39] - w[40] - w[41] - w[42] + 61

* **Constraints**

In [72]:
@contraints(model, begin
        [i in V], sum(x[i,k] for k in C) == 1
        [k in C], sum(a[i]*x[i,k] for i in V) >= (1-alpha)*M[k]
        [k in C], sum(a[i]*x[i,k] for i in V) <= (1+alpha)*M[k]
        
        end)

* **Running the model**

* **Printing variables**

In [ ]:
println("Objective function = ", JuMP.objective_value(model))
for k in C
    print("Group ", k, ": ")
    aux = 0 # Auxiliary variable that counts the group load
    cnt = 0 # Counts the number of objects in each group
    for i in V
        if(value(x[i,k]) >= 0.9)
            print(i, " ")
            aux += a[i]
            cnt += 1
        end
    end
    println("(", aux, ") (", cnt, " objects)")
end

## References

Baker, K. R., & Powell, S. G. (2002). Methods for assigning students to groups: A study of alternative objective functions. Journal of the Operational Research Society, 53(4), 397-404.

Fernández, E., Kalcsics, J., & Nickel, S. (2013). The maximum dispersion problem. Omega, 41(4), 721-730.

Likert, R. (1932). A technique for the measurement of attitudes. Archives of psychology.